In [37]:
import pandas as pd

In [38]:
tk_data=pd.read_csv('/home/thiri/Documents/RT Projects/nov_april/filtered_dataset/rt_data_filtered_columns.csv')

In [39]:
lt_data=pd.read_csv('/home/thiri/Documents/RT Projects/nov_april/agg/lt_data.csv')

In [40]:
plan_summary=pd.read_csv('/home/thiri/Documents/RT Projects/nov_april/filtered_dataset/plan_summary_filtered_columns_1.csv')

In [41]:
plan_summary.shape

(961708, 3)

### **Loyalty Duration**

In [42]:
tk_data['Date Created'] = pd.to_datetime(tk_data['Date Created'], errors='coerce')

# Merge to bring 'Date Created' into updated_tkOpen_count_data
merged_data = pd.merge(
    lt_data, 
    tk_data[['Customer ID', 'Date Created']], 
    on=['Customer ID'], 
    how='left'
)

# Get the latest (max) date per 'Customer ID', 'Local Service ID', 'Cpe ID'
latest_date_data = (
    merged_data.groupby(['Customer ID'], as_index=False)
    .agg({'Date Created': 'max'})  # Ensures the latest date is selected
)

# Merge back to assign 'Last Ticket Created Date'
last_tk_created_date = pd.merge(
    lt_data, 
    latest_date_data, 
    on=['Customer ID'], 
    how='left'
)

# Rename column for clarity
last_tk_created_date.rename(columns={'Date Created': 'Last Ticket Created Date'}, inplace=True)

last_tk_created_date

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2275947.0                        0   
1                1    2150547.0                        2   
2                2    2178246.0                        0   
3                3    2347542.0                        0   
4                4    2141477.0                        2   
...            ...          ...                      ...   
178716      178716    2576650.0                        0   
178717      178717    2452503.0                        1   
178718      178718    2538172.0                        1   
178719      178719    2487901.0                        0   
178720      178720    2117914.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             0                              0           Low   
1                             2                              0           Low   
2                             0                              1        Medium   
3                             2                              0           Low   
4                             3                              5           Low   
...                         ...                            ...           ...   
178716                        0                              0           Low   
178717                        1                              0          High   
178718                        1                              0          High   
178719                        0                              0           Low   
178720                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  
0        1.166447           1.166447       1.166447               2024-11-10  
1        1.796053           1.522416       1.983368               2025-04-07  
2        2.943935          26.436464      49.928993               2025-04-29  
3        5.231470           2.409657       5.231470               2025-03-20  
4        1.687940           3.703595      28.528472               2025-04-20  
...           ...                ...            ...                      ...  
178716   2.460000           2.460000       2.460000               2025-04-28  
178717  23.990000          23.990000      23.990000               2025-04-03  
178718   2.460000           2.460000       2.460000               2025-04-12  
178719  32.770000          32.770000      32.770000               2025-04-28  
178720   2.460000           2.460000       2.460000               2025-04-06  

[178721 rows x 10 columns]

### Check if 'Last Ticket Created Date' is assigned correctly or not

In [43]:
tk_data[tk_data['Customer ID']==2206790][['Customer ID','Date Created']]

Customer ID Date Created
344252    2206790.0   2025-03-01

### Assign Plan Start Date

In [44]:
summary=plan_summary[['Customer ID','Plan Start Date']]

In [45]:
if 'Plan Start Date' in last_tk_created_date.columns:
    last_tk_created_date = last_tk_created_date.drop(columns=['Plan Start Date'])

# Keep only the earliest 'Plan Start Date' for each unique 'Customer ID'
latest_plan_start = summary[['Customer ID','Plan Start Date']].dropna()
latest_plan_start = latest_plan_start.sort_values(by=['Customer ID', 'Plan Start Date']).drop_duplicates(subset=['Customer ID'], keep='first')

# Merge
plan_start_date_data = last_tk_created_date.merge(
    latest_plan_start,
    on=['Customer ID'],
    how='left'
)

plan_start_date_data

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2275947.0                        0   
1                1    2150547.0                        2   
2                2    2178246.0                        0   
3                3    2347542.0                        0   
4                4    2141477.0                        2   
...            ...          ...                      ...   
178716      178716    2576650.0                        0   
178717      178717    2452503.0                        1   
178718      178718    2538172.0                        1   
178719      178719    2487901.0                        0   
178720      178720    2117914.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             0                              0           Low   
1                             2                              0           Low   
2                             0                              1        Medium   
3                             2                              0           Low   
4                             3                              5           Low   
...                         ...                            ...           ...   
178716                        0                              0           Low   
178717                        1                              0          High   
178718                        1                              0          High   
178719                        0                              0           Low   
178720                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        1.166447           1.166447       1.166447               2024-11-10   
1        1.796053           1.522416       1.983368               2025-04-07   
2        2.943935          26.436464      49.928993               2025-04-29   
3        5.231470           2.409657       5.231470               2025-03-20   
4        1.687940           3.703595      28.528472               2025-04-20   
...           ...                ...            ...                      ...   
178716   2.460000           2.460000       2.460000               2025-04-28   
178717  23.990000          23.990000      23.990000               2025-04-03   
178718   2.460000           2.460000       2.460000               2025-04-12   
178719  32.770000          32.770000      32.770000               2025-04-28   
178720   2.460000           2.460000       2.460000               2025-04-06   

       Plan Start Date  
0           2021-05-22  
1           2020-02-02  
2           2020-06-09  
3           2021-12-08  
4           2019-12-29  
...                ...  
178716      2025-02-12  
178717      2023-01-12  
178718      2024-07-17  
178719      2023-09-18  
178720      2019-09-05  

[178721 rows x 11 columns]

In [46]:
plan_start_date_data['Plan Start Date'].isna().value_counts()

Plan Start Date
False    178719
True          2
Name: count, dtype: int64

In [47]:
plan_start_date_data[plan_start_date_data['Plan Start Date'].isna()]

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
88126        88126    2551730.0                        0   
175401      175401    2100736.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
88126                         0                              0           Low   
175401                        1                              1           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
88126    0.986667           0.986667       0.986667               2025-01-26   
175401   9.570000           9.570000       9.570000               2025-04-09   

       Plan Start Date  
88126              NaN  
175401             NaN

In [48]:
tk_data[tk_data['Customer ID']==2100736]

Unnamed: 0  Customer ID     CPE ID  Ticket ID Date Created  \
407908       65916    2100736.0  FFG008371    7588127   2025-04-09   

                    Started Closed Resolved         Root Cause Priority  \
407908  2025-04-24T12:13:26    NaN      NaN  Termination Saved      Low   

       Plan Start Date       Status Life Time Service Time  Fiber Olt Name  \
407908      2024-09-21  in_progress       NaN          NaN  OVF-PNIX07ADAY   

          Ticket Problem  
407908  Customer Request

### Calculate Loyalty Duration

In [49]:
plan_start_date_data['Plan Start Date'] = pd.to_datetime(plan_start_date_data['Plan Start Date'], errors='coerce')
plan_start_date_data['Last Ticket Created Date'] = pd.to_datetime(plan_start_date_data['Last Ticket Created Date'], errors='coerce')

plan_start_date_data['Loyalty Duration'] = (
    ((plan_start_date_data['Last Ticket Created Date'] - plan_start_date_data['Plan Start Date']).dt.days / 30.44)*30
).round(2) 

plan_start_date_data

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2275947.0                        0   
1                1    2150547.0                        2   
2                2    2178246.0                        0   
3                3    2347542.0                        0   
4                4    2141477.0                        2   
...            ...          ...                      ...   
178716      178716    2576650.0                        0   
178717      178717    2452503.0                        1   
178718      178718    2538172.0                        1   
178719      178719    2487901.0                        0   
178720      178720    2117914.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             0                              0           Low   
1                             2                              0           Low   
2                             0                              1        Medium   
3                             2                              0           Low   
4                             3                              5           Low   
...                         ...                            ...           ...   
178716                        0                              0           Low   
178717                        1                              0          High   
178718                        1                              0          High   
178719                        0                              0           Low   
178720                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        1.166447           1.166447       1.166447               2024-11-10   
1        1.796053           1.522416       1.983368               2025-04-07   
2        2.943935          26.436464      49.928993               2025-04-29   
3        5.231470           2.409657       5.231470               2025-03-20   
4        1.687940           3.703595      28.528472               2025-04-20   
...           ...                ...            ...                      ...   
178716   2.460000           2.460000       2.460000               2025-04-28   
178717  23.990000          23.990000      23.990000               2025-04-03   
178718   2.460000           2.460000       2.460000               2025-04-12   
178719  32.770000          32.770000      32.770000               2025-04-28   
178720   2.460000           2.460000       2.460000               2025-04-06   

       Plan Start Date  Loyalty Duration  
0           2021-05-22           1249.67  
1           2020-02-02           1863.67  
2           2020-06-09           1759.20  
3           2021-12-08           1180.68  
4           2019-12-29           1910.97  
...                ...               ...  
178716      2025-02-12             73.92  
178717      2023-01-12            800.26  
178718      2024-07-17            265.11  
178719      2023-09-18            579.50  
178720      2019-09-05           2010.51  

[178721 rows x 12 columns]

## **On Going Ticket**

In [50]:
tk_data['Status'].value_counts()

Status
closed         387177
in_progress     20991
pending          3363
new              2072
re-open           955
resolved          703
re-open-2         214
resolved-2        147
re-open-3          74
resolved-3         25
re-open-4          13
resolved-4         10
resolved-5          3
Name: count, dtype: int64

In [51]:
status_closed=tk_data[tk_data['Status']=='closed']
status_closed.shape

(387177, 16)

In [52]:
status_others=tk_data[tk_data['Status'].isin(['in_progress','new','resolved','pending','re-open','resolved-2','re-open-2','re-open-3','re-open-4','resolved-3','resolved-4','resolved-5'])]
status_others.shape

(28570, 16)

In [53]:
on_going_tks = plan_start_date_data.merge(
    status_others[['Customer ID']].drop_duplicates(),
    left_on=['Customer ID'], 
    right_on=['Customer ID'], 
    how='left',
    indicator=True
)

on_going_tks['On Going Ticket'] = (on_going_tks['_merge'] == 'both').astype(int)
on_going_tks.drop(columns=['_merge'], inplace=True)


In [54]:
on_going_tks

Unnamed: 0  Customer ID  Tk Count Within 30 Days  \
0                0    2275947.0                        0   
1                1    2150547.0                        2   
2                2    2178246.0                        0   
3                3    2347542.0                        0   
4                4    2141477.0                        2   
...            ...          ...                      ...   
178716      178716    2576650.0                        0   
178717      178717    2452503.0                        1   
178718      178718    2538172.0                        1   
178719      178719    2487901.0                        0   
178720      178720    2117914.0                        1   

        Tk Count Within 60 Days  Previous Termination Attempts Last Priority  \
0                             0                              0           Low   
1                             2                              0           Low   
2                             0                              1        Medium   
3                             2                              0           Low   
4                             3                              5           Low   
...                         ...                            ...           ...   
178716                        0                              0           Low   
178717                        1                              0          High   
178718                        1                              0          High   
178719                        0                              0           Low   
178720                        1                              0           Low   

        Life Time  Average Life Time  Max Life Time Last Ticket Created Date  \
0        1.166447           1.166447       1.166447               2024-11-10   
1        1.796053           1.522416       1.983368               2025-04-07   
2        2.943935          26.436464      49.928993               2025-04-29   
3        5.231470           2.409657       5.231470               2025-03-20   
4        1.687940           3.703595      28.528472               2025-04-20   
...           ...                ...            ...                      ...   
178716   2.460000           2.460000       2.460000               2025-04-28   
178717  23.990000          23.990000      23.990000               2025-04-03   
178718   2.460000           2.460000       2.460000               2025-04-12   
178719  32.770000          32.770000      32.770000               2025-04-28   
178720   2.460000           2.460000       2.460000               2025-04-06   

       Plan Start Date  Loyalty Duration  On Going Ticket  
0           2021-05-22           1249.67                0  
1           2020-02-02           1863.67                0  
2           2020-06-09           1759.20                0  
3           2021-12-08           1180.68                0  
4           2019-12-29           1910.97                0  
...                ...               ...              ...  
178716      2025-02-12             73.92                1  
178717      2023-01-12            800.26                1  
178718      2024-07-17            265.11                1  
178719      2023-09-18            579.50                1  
178720      2019-09-05           2010.51                1  

[178721 rows x 13 columns]

In [55]:
on_going_tks['On Going Ticket'].value_counts()

On Going Ticket
0    150430
1     28291
Name: count, dtype: int64

In [57]:
on_going_tks.to_csv('agg/prepared_data.csv')